# 简单跑点代码
## 1. 本地跑个BLIP模型做tool

In [1]:
from agentlego import list_tools, load_tool

In [2]:
print(list_tools())  # list tools in AgentLego

['Calculator', 'CannyTextToImage', 'ImageToCanny', 'DepthTextToImage', 'ImageToDepth', 'ImageExpansion', 'ImageStylization', 'ObjectRemove', 'ObjectReplace', 'HumanBodyPose', 'HumanFaceLandmark', 'PoseToImage', 'ImageToScribble', 'ScribbleTextToImage', 'ImageCaption', 'TextToImage', 'AudioImageToImage', 'AudioTextToImage', 'AudioToImage', 'ThermalToImage', 'ObjectDetection', 'TextToBbox', 'OCR', 'GoogleSearch', 'SegmentAnything', 'SegmentObject', 'SemanticSegmentation', 'SpeechToText', 'TextToSpeech', 'Translation', 'VisualQuestionAnswering']


In [3]:
image_caption_tool = load_tool('ImageCaption', device='cuda')
print(image_caption_tool.description)
image = './examples/demo.png'

caption = image_caption_tool(image)

A useful tool that returns a brief description of the input image. Args: image (image path)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'BlipTokenizer'.


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmclassification/v1/blip/blip-base_3rdparty_coco-caption_20230419-a5b71af3.pth


Output()

这个是被BLIP“解释”的图片
![](./demo.png)
下面是BLIP的输出结果

In [4]:
caption

'a puppy and a cat sitting on a blanket'

## 2. 简单跑个计算器的tool

In [5]:
from agentlego.apis import load_tool

# load tool
tool = load_tool('Calculator')

# apply tool
tool('e ** cos(pi)')

'0.36787944117144233'

## 2.1 导入OpenAI的KEY

In [6]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxx'

## 3. 将agentlego的计算器转换为lagent的工具

In [7]:
from agentlego.apis import load_tool
from lagent import ReAct, GPTAPI, ActionExecutor

# load tools and build agent
# please set `OPENAI_API_KEY` in your environment variable.
tool = load_tool('Calculator').to_lagent()
agent = ReAct(GPTAPI(temperature=0.), action_executor=ActionExecutor([tool]))

# agent running with the tool.
ret = agent.chat(f'pi and 3.2, which is greater?')
for step in ret.inner_steps[1:]:
    print('------')
    print(step['content'])

------
Thought: To determine which number is greater, we can compare their values.

Action: Calculator
Action Input: {"expression": "pi > 3.2"}
------
Response:False

------
Thought: The result of the comparison is False, which means that 3.2 is greater than pi.

Final Answer: 3.2 is greater than pi.


## 4. 将agentlego的计算器转换为langchain的工具

In [8]:
from agentlego.apis import load_tool
from langchain.agents import AgentType, initialize_agent
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

# 加载要使用的工具。
tool = load_tool('Calculator').to_langchain()

# 构建 LangChain 智能体链
model = ChatOpenAI(temperature=0., model='gpt-4')
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
agent = initialize_agent(
   agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
   llm=model,
   tools=[tool],
   memory=memory,
   verbose=True,
)

user_input = '如果三角形的边长分别为3cm、4cm和5cm，请用工具计算三角形的面积。'
agent.run(input=user_input)



> Entering new AgentExecutor chain...
The area of a triangle can be calculated using Heron's formula, which is sqrt[s(s-a)(s-b)(s-c)], where s is the semi-perimeter of the triangle, and a, b, and c are the sides of the triangle. In this case, a=3cm, b=4cm, c=5cm, and s=(a+b+c)/2=6cm. So, the area of the triangle is sqrt[6(6-3)(6-4)(6-5)]. Let's calculate this. 

Action:
```
{
  "action": "Calculator",
  "action_input": {
    "expression": "((6*(6-3)*(6-4)*(6-5))**0.5)"
  }
}
```
Observation: 6.0
Thought:The calculation result is 6.0, which means the area of the triangle is 6 square centimeters. Now I will provide this answer to the user. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "这个三角形的面积是6平方厘米。"
}
```

> Finished chain.


'这个三角形的面积是6平方厘米。'